# Trade Strong Price Deviations on Faster Spot Markets by Supplying Demand on Slower Executing Markets

The premise of this bot is to establish a potential arbitrage opportunity that exists when slowly executing markets, e.g. those requiring bank transfers deviate too strongly from other spot markets. We could therefore buy on one, sell on the other and make profit.

For this reason we need to do:

- Get the price data from one spot market, e.g. Binance BTC/EUR
- Get the price data from the slow market, e.g. Bitcoin.de
- Constantly measure price deviation
- And then if too high, put an offer on the one, and bid on the other
- Repeat as long as there are funds

In [5]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

# Define the API endpoints for bitcoin.de and Bybit
bitcoin_de_key = "baee5b0755b0902a1e25a5e545a411c2"
bitcoin_de_pair = "btceur"
bitcoin_de_url = f"https://api.bitcoin.de/v4/{bitcoin_de_pair}/basic/rate.json?apikey={bitcoin_de_key}"


# Fetch BTC prices
response_bitcoin_de = requests.get(bitcoin_de_url)

data_bitcoin_de = response_bitcoin_de.json()

# price_bitcoin_de = data_bitcoin_de['last']
print(data_bitcoin_de)


{'rate': {'rate_weighted': '61316.65000000', 'rate_weighted_3h': '61156.90961587', 'rate_weighted_12h': '61629.90826065'}, 'errors': [], 'credits': 20}


In [ ]:
import ccxt
import pandas as pd

# Create an exchange object for Binance
exchange = ccxt.binance()

# Define the trading pair (e.g., BTC/USDT)
trading_pair = 'BTC/USDT'

# Fetch real-time price data
data = exchange.fetch_ticker(trading_pair)

# Convert the data to a Pandas dataframe
df_binance = pd.DataFrame([data], columns=['symbol', 'timestamp', 'last'])
df_binance['timestamp'] = pd.to_datetime(df_binance['timestamp'], unit='ms')

# Load the previously saved bitcoin.de dataframe
df_bitcoin_de = pd.read_csv('bitcoin_prices.csv')

# Merge the dataframes based on timestamp
df_combined = pd.merge(df_bitcoin_de, df_binance, on='timestamp', suffixes=('_bitcoin_de', '_binance'))

# Save the combined dataframe to a CSV file
df_combined.to_csv('combined_prices.csv', index=False)

print(f"Combined Bitcoin price data saved to 'combined_prices.csv'.")


In [ ]:
import ccxt
import pandas as pd

# Create exchange objects for bitcoin.de and Bybit
exchange_bitcoin_de = ccxt.bitcoinde()
exchange_bybit = ccxt.bybit()

# Define the trading pair (e.g., BTC/EUR)
trading_pair = 'BTC/EUR'

# Fetch BTC prices
price_bitcoin_de = exchange_bitcoin_de.fetch_ticker(trading_pair)['last']
price_bybit = exchange_bybit.fetch_ticker('BTC/USD')['last']

# Calculate the price difference
price_difference = price_bitcoin_de - price_bybit

# Create a dataframe
df = pd.DataFrame({
    'Exchange': ['bitcoin.de', 'Bybit'],
    'BTC Price': [price_bitcoin_de, price_bybit],
    'Price Difference': [price_difference, -price_difference]  # Negative value for Bybit
})

# Save the dataframe to a CSV file
df.to_csv('btc_price_difference.csv', index=False)

print(f"BTC price difference saved to 'btc_price_difference.csv'.")
